In [1]:
import pandas as pd
import backtesting
# from backtesting import Backtest, Strategy
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import *
import numpy as np
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

import ipywidgets as widgets

C:\Users\Karolina\anaconda3\envs\Magisterka\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
data = pd.read_csv('data/decisions_EURUSD_5Y.csv')

In [6]:
data.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,Name,Return,Up,...,MACD_decision,diff_MACD,diff_MACD_prev,fuzz_MACD_strategy,dec_bollinger,diff_cl_BU,diff_cl_BL,prev_diff_cl_BU,prev_diff_cl_BL,fuzz_BB_strategy
0,2019-05-02,1.120185,1.121818,1.117356,1.120084,1.120084,0,EURUSD=X,-0.001356,0.000000,...,0,0.000048,0.000000,0,0,-0.013554,0.006675,-0.012144,0.007844,0
1,2019-05-03,1.117256,1.120072,1.114132,1.117356,1.117356,0,EURUSD=X,-0.002439,0.000000,...,-1,-0.000081,0.000048,0,0,-0.016387,0.004574,-0.013554,0.006675,0
2,2019-05-06,1.118706,1.120448,1.117993,1.118581,1.118581,0,EURUSD=X,0.001096,0.001096,...,0,-0.000063,-0.000081,0,0,-0.015189,0.006163,-0.016387,0.004574,0
3,2019-05-07,1.120448,1.121957,1.117606,1.120185,1.120185,0,EURUSD=X,0.001433,0.001433,...,1,0.000072,-0.000063,0,0,-0.013316,0.008045,-0.015189,0.006163,0
4,2019-05-08,1.118956,1.121500,1.118956,1.119107,1.119107,0,EURUSD=X,-0.000963,0.000000,...,0,0.000100,0.000072,0,0,-0.013974,0.007306,-0.013316,0.008045,0
5,2019-05-09,1.119746,1.125100,1.117568,1.119833,1.119833,0,EURUSD=X,0.000649,0.000649,...,0,0.000176,0.000100,0,0,-0.012623,0.008189,-0.013974,0.007306,0
6,2019-05-10,1.121856,1.125239,1.121856,1.122208,1.122208,0,EURUSD=X,0.002119,0.002119,...,0,0.000382,0.000176,0,0,-0.009882,0.010585,-0.012623,0.008189,0
7,2019-05-13,1.123671,1.126342,1.122410,1.123646,1.123646,0,EURUSD=X,0.001280,0.001280,...,0,0.000599,0.000382,0,0,-0.007310,0.011574,-0.009882,0.010585,0
8,2019-05-14,1.123242,1.124606,1.120486,1.123255,1.123255,0,EURUSD=X,-0.000348,0.000000,...,0,0.000692,0.000599,0,0,-0.006398,0.010604,-0.007310,0.011574,0
9,2019-05-15,1.120360,1.122272,1.118043,1.120448,1.120448,0,EURUSD=X,-0.002502,0.000000,...,0,0.000548,0.000692,0,0,-0.008067,0.007465,-0.006398,0.010604,0


In [7]:
data.columns.tolist()

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'Name',
 'Return',
 'Up',
 'Down',
 'avg_14up',
 'avg_14down',
 'RS_14',
 'RSI',
 '5MA',
 '15MA',
 '12Ewm',
 '26Ewm',
 'MACD',
 'low_14',
 'high_14',
 'k_percent',
 'r_percent',
 'B_MA',
 'BU',
 'BL',
 'dec_rsi',
 'Signal_RSI_fuzz_system',
 'dec_will',
 'Signal_william_fuzz_system',
 'prev_5MA',
 'crossover',
 'cross_MA',
 'diff_MA',
 'diff_MA_prev',
 'fuzz_MA_strategy',
 'Signal_Line',
 'MACD_decision',
 'diff_MACD',
 'diff_MACD_prev',
 'fuzz_MACD_strategy',
 'dec_bollinger',
 'diff_cl_BU',
 'diff_cl_BL',
 'prev_diff_cl_BU',
 'prev_diff_cl_BL',
 'fuzz_BB_strategy']

### Sprawdzanie sygnałów w kolumnach ostrych i rozmytych

In [8]:
# Zliczenie przypadków, w których wartości są przeciwne

rsi_count = ((data['dec_rsi'] == 1) & (data['Signal_RSI_fuzz_system'] == -1) | (data['dec_rsi'] == -1) & (data['Signal_RSI_fuzz_system'] == 1)).sum()
print("rsi_count", rsi_count)

williams_count = ((data['dec_will'] == 1) & (data['Signal_william_fuzz_system'] == -1) | (data['dec_will'] == -1) & (data['Signal_william_fuzz_system'] == 1)).sum()
print("williams_count", williams_count)

# cross_MA - ostre decyzje
ma_count = ((data['cross_MA'] == 1) & (data['fuzz_MA_strategy'] == -1) | (data['cross_MA'] == -1) & (data['fuzz_MA_strategy'] == 1)).sum()
print("ma_count", ma_count)

macd_count = ((data['MACD_decision'] == 1) & (data['fuzz_MACD_strategy'] == -1) | (data['MACD_decision'] == -1) & (data['fuzz_MACD_strategy'] == 1)).sum()
print("macd_count", macd_count)


bb_count = ((data['dec_bollinger'] == 1) & (data['fuzz_BB_strategy'] == -1) | (data['dec_bollinger'] == -1) & (data['fuzz_BB_strategy'] == 1)).sum()

print("bb_count", bb_count)


rsi_count 0
williams_count 0
ma_count 1
macd_count 0
bb_count 0


In [9]:
opposite_rows = data[((data['dec_rsi'] == 1) & (data['Signal_RSI_fuzz_system'] == -1)) | ((data['dec_rsi'] == -1) & (data['Signal_RSI_fuzz_system'] == 1))]
opposite_rows[['Date','dec_rsi', 'Signal_RSI_fuzz_system', 'RS_14', 'RSI']]


,Date,dec_rsi,Signal_RSI_fuzz_system,RS_14,RSI


In [10]:
opposite_rows = data[((data['cross_MA'] == 1) & (data['fuzz_MA_strategy'] == -1)) | ((data['cross_MA'] == -1) & (data['fuzz_MA_strategy'] == 1))]
opposite_rows[['Date', '5MA', '15MA', 'cross_MA', 'fuzz_MA_strategy', 'diff_MA', 'diff_MA_prev']]


,Date,5MA,15MA,cross_MA,fuzz_MA_strategy,diff_MA,diff_MA_prev
251,2020-04-20,1.090778,1.090844,-1,1,-0.000066,-0.000566


<img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExNWcyanR3eWUxYTRsNHpyZTAxbmoyNHcxOTJnbWtqcnl5NHRyN2hqMCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/jPAdK8Nfzzwt2/giphy.gif" width="350" align="center">

Na ten moment nie wiem czemu tutaj jest rozbieżność

In [3]:
data = data.set_index(pd.DatetimeIndex(data['Date']))

## Strategia z tp i sl

### Ostre

In [32]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['dec_rsi'], 'rsi')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
# stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
#            ssl=[0.01, 0.1],
#            size_tr=[0.4, 1], 
#            maximize='Equity Final [$]',
#            method='skopt',
#            max_tries=1000,
#            random_state=10,
#            return_heatmap=True,
#            return_optimization=True)
# heatmap.sort_values().iloc[-3:]

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   95.482866
Equity Final [$]                  8853.349503
Equity Peak [$]                  10348.859286
Return [%]                         -11.466505
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -2.361916
Volatility (Ann.) [%]                5.091724
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -17.931377
Avg. Drawdown [%]                   -3.085824
Max. Drawdown Duration     1465 days 00:00:00
Avg. Drawdown Duration      250 days 00:00:00
# Trades                                   51
Win Rate [%]                        60.784314
Best Trade [%]                       3.976295
Worst Trade [%]                     -5.146269
Avg. Trade [%]                    

In [13]:
stats_skopt

NameError: name 'stats_skopt' is not defined

### Rozmyte

In [33]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['Signal_RSI_fuzz_system'], 'rsi')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
# stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
#            ssl=[0.01, 0.1],
#            size_tr=[0.4, 1], 
#            maximize='Equity Final [$]',
#            method='skopt',
#            max_tries=1000,
#            random_state=10,
#            return_heatmap=True,
#            return_optimization=True)
# heatmap.sort_values().iloc[-3:]

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   97.196262
Equity Final [$]                  9158.559619
Equity Peak [$]                  10341.512101
Return [%]                          -8.414404
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -1.710271
Volatility (Ann.) [%]                4.553984
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -15.155075
Avg. Drawdown [%]                   -3.086583
Max. Drawdown Duration     1475 days 00:00:00
Avg. Drawdown Duration      251 days 00:00:00
# Trades                                   71
Win Rate [%]                        69.014085
Best Trade [%]                       3.167082
Worst Trade [%]                     -5.537888
Avg. Trade [%]                    

In [15]:
stats_skopt

NameError: name 'stats_skopt' is not defined

***
***

## Williams %R

### Ostre

In [34]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['dec_will'], 'williams')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
# stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
#            ssl=[0.01, 0.1],
#            size_tr=[0.4, 1], 
#            maximize='Equity Final [$]',
#            method='skopt',
#            max_tries=1000,
#            random_state=10,
#            return_heatmap=True,
#            return_optimization=True)
# heatmap.sort_values().iloc[-3:]

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   99.299065
Equity Final [$]                  9133.660353
Equity Peak [$]                  10429.950521
Return [%]                          -8.663396
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -1.762773
Volatility (Ann.) [%]                3.984546
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -13.038831
Avg. Drawdown [%]                   -1.525769
Max. Drawdown Duration     1465 days 00:00:00
Avg. Drawdown Duration      162 days 00:00:00
# Trades                                  102
Win Rate [%]                        56.862745
Best Trade [%]                       4.541013
Worst Trade [%]                     -5.450947
Avg. Trade [%]                    

In [17]:
stats_skopt

NameError: name 'stats_skopt' is not defined

### Rozmyte

In [35]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['Signal_william_fuzz_system'], 'williams')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
# stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
#            ssl=[0.01, 0.1],
#            size_tr=[0.4, 1], 
#            maximize='Equity Final [$]',
#            method='skopt',
#            max_tries=1000,
#            random_state=10,
#            return_heatmap=True,
#            return_optimization=True)
# heatmap.sort_values().iloc[-3:]

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   99.299065
Equity Final [$]                   9655.11586
Equity Peak [$]                  10489.823477
Return [%]                          -3.448841
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -0.686456
Volatility (Ann.) [%]                3.720288
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -8.845585
Avg. Drawdown [%]                   -1.731534
Max. Drawdown Duration      740 days 00:00:00
Avg. Drawdown Duration      148 days 00:00:00
# Trades                                  121
Win Rate [%]                        51.239669
Best Trade [%]                       4.190261
Worst Trade [%]                     -4.497805
Avg. Trade [%]                    

In [18]:
stats_skopt

NameError: name 'stats_skopt' is not defined

## MA

### Ostre

In [36]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['cross_MA'], 'ma')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
# stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
#            ssl=[0.01, 0.1],
#            size_tr=[0.4, 1], 
#            maximize='Equity Final [$]',
#            method='skopt',
#            max_tries=1000,
#            random_state=10,
#            return_heatmap=True,
#            return_optimization=True)
# heatmap.sort_values().iloc[-3:]

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   92.133956
Equity Final [$]                  8297.933991
Equity Peak [$]                  10094.161223
Return [%]                          -17.02066
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -3.595588
Volatility (Ann.) [%]                3.961987
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.855642
Avg. Drawdown [%]                   -5.700589
Max. Drawdown Duration     1744 days 00:00:00
Avg. Drawdown Duration      447 days 00:00:00
# Trades                                   57
Win Rate [%]                        29.824561
Best Trade [%]                        5.72109
Worst Trade [%]                     -5.790727
Avg. Trade [%]                    

In [20]:
stats_skopt

NameError: name 'stats_skopt' is not defined

### Rozmyte

In [37]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['fuzz_MA_strategy'], 'ma')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
# stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
#            ssl=[0.01, 0.1],
#            size_tr=[0.4, 1], 
#            maximize='Equity Final [$]',
#            method='skopt',
#            max_tries=1000,
#            random_state=10,
#            return_heatmap=True,
#            return_optimization=True)
# heatmap.sort_values().iloc[-3:]

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   77.258567
Equity Final [$]                  8555.890638
Equity Peak [$]                  10283.944558
Return [%]                         -14.441094
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -3.014623
Volatility (Ann.) [%]                4.086485
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -17.021116
Avg. Drawdown [%]                   -3.044238
Max. Drawdown Duration     1475 days 00:00:00
Avg. Drawdown Duration      198 days 00:00:00
# Trades                                   24
Win Rate [%]                        33.333333
Best Trade [%]                        4.79884
Worst Trade [%]                     -5.165642
Avg. Trade [%]                    

In [22]:
stats_skopt

NameError: name 'stats_skopt' is not defined

## MACD

### Ostre

In [38]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['MACD_decision'], 'macd')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
# stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
#            ssl=[0.01, 0.1],
#            size_tr=[0.4, 1], 
#            maximize='Equity Final [$]',
#            method='skopt',
#            max_tries=1000,
#            random_state=10,
#            return_heatmap=True,
#            return_optimization=True)
# heatmap.sort_values().iloc[-3:]

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   93.380062
Equity Final [$]                  8902.022633
Equity Peak [$]                  10013.471809
Return [%]                         -10.979774
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -2.256797
Volatility (Ann.) [%]                4.307817
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -17.815738
Avg. Drawdown [%]                      -6.251
Max. Drawdown Duration     1769 days 00:00:00
Avg. Drawdown Duration      599 days 00:00:00
# Trades                                   74
Win Rate [%]                        35.135135
Best Trade [%]                       5.147126
Worst Trade [%]                     -5.147191
Avg. Trade [%]                    

In [23]:
stats_skopt

NameError: name 'stats_skopt' is not defined

In [39]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['fuzz_MACD_strategy'], 'macd')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
# stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
#            ssl=[0.01, 0.1],
#            size_tr=[0.4, 1], 
#            maximize='Equity Final [$]',
#            method='skopt',
#            max_tries=1000,
#            random_state=10,
#            return_heatmap=True,
#            return_optimization=True)
# heatmap.sort_values().iloc[-3:]

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   85.046729
Equity Final [$]                  8636.181092
Equity Peak [$]                  10417.033319
Return [%]                         -13.638189
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -2.836668
Volatility (Ann.) [%]                5.206625
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -21.977204
Avg. Drawdown [%]                   -2.503219
Max. Drawdown Duration     1440 days 00:00:00
Avg. Drawdown Duration      142 days 00:00:00
# Trades                                   28
Win Rate [%]                        53.571429
Best Trade [%]                       5.411858
Worst Trade [%]                     -5.063979
Avg. Trade [%]                    

In [25]:
stats_skopt

NameError: name 'stats_skopt' is not defined

## Bollinger Bands 

### Ostre

In [5]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.047
    ttp = 0.051
    size_tr = 0.95
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['dec_bollinger'], 'bollinger')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
# stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
#            ssl=[0.01, 0.1],
#            size_tr=[0.4, 1], 
#            maximize='Equity Final [$]',
#            method='skopt',
#            max_tries=1000,
#            random_state=10,
#            return_heatmap=True,
#            return_optimization=True)
# heatmap.sort_values().iloc[-3:]

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   96.105919
Equity Final [$]                 10413.648673
Equity Peak [$]                  10976.506759
Return [%]                           4.136487
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                    0.798665
Volatility (Ann.) [%]                5.341913
Sharpe Ratio                         0.149509
Sortino Ratio                        0.216883
Calmar Ratio                         0.071052
Max. Drawdown [%]                  -11.240623
Avg. Drawdown [%]                   -1.528137
Max. Drawdown Duration      673 days 00:00:00
Avg. Drawdown Duration       88 days 00:00:00
# Trades                                   57
Win Rate [%]                        71.929825
Best Trade [%]                       5.696024
Worst Trade [%]                     -4.853524
Avg. Trade [%]                    

In [27]:
stats_skopt

NameError: name 'stats_skopt' is not defined

### Rozmyte

In [7]:
class CustomStrategy(SignalStrategy):
# 0.063006 0.099444 0.999104   
    ssl = 0.099444
    ttp = 0.063006
    size_tr = 0.999104
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['fuzz_BB_strategy'], 'bollinger')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
# stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
#            ssl=[0.01, 0.1],
#            size_tr=[0.4, 1], 
#            maximize='Equity Final [$]',
#            method='skopt',
#            max_tries=1000,
#            random_state=10,
#            return_heatmap=True,
#            return_optimization=True)
# heatmap.sort_values().iloc[-3:]

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   97.196262
Equity Final [$]                 14109.246105
Equity Peak [$]                  14494.580787
Return [%]                          41.092461
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                    6.989675
Volatility (Ann.) [%]                6.228576
Sharpe Ratio                         1.122195
Sortino Ratio                        1.850863
Calmar Ratio                         1.002567
Max. Drawdown [%]                    -6.97178
Avg. Drawdown [%]                   -0.835278
Max. Drawdown Duration      203 days 00:00:00
Avg. Drawdown Duration       23 days 00:00:00
# Trades                                   33
Win Rate [%]                        90.909091
Best Trade [%]                       7.340569
Worst Trade [%]                      -3.34573
Avg. Trade [%]                    

In [29]:
stats_skopt

NameError: name 'stats_skopt' is not defined